In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Keras
import keras

In [3]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [5]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [6]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00056.wav,0.391059,0.137728,1811.076084,2052.332563,3927.809582,0.075480,-145.434570,102.829018,-12.517674,...,7.457216,-10.470443,-2.360484,-6.783623,2.671134,-4.760879,-0.949005,0.024832,-2.005315,blues
1,blues.00082.wav,0.338896,0.251350,2141.461656,2168.015560,4627.997015,0.105151,-29.362093,108.667946,-25.573164,...,5.456504,-7.687712,7.410599,-11.319178,7.229288,-9.466551,1.930059,-6.328475,-1.304813,blues
2,blues.00073.wav,0.341865,0.259714,2094.919420,2158.335963,4482.447895,0.100570,-30.094570,103.732292,-19.754238,...,11.213472,-8.842510,9.090302,-3.579252,4.372715,-2.830806,4.725090,0.485058,1.475955,blues
3,blues.00006.wav,0.291308,0.141796,1459.078483,1388.913312,2795.616429,0.073028,-190.149460,130.296951,-36.344139,...,-3.125678,-6.593118,-9.942267,-6.537224,-10.064754,-10.912171,-6.972478,-3.449033,-6.495510,blues
4,blues.00076.wav,0.365575,0.219080,2645.681661,2426.198512,5612.507729,0.136469,-52.693062,74.430283,-18.692436,...,7.792499,-10.561926,6.946956,-9.127866,13.020091,-9.854002,3.689460,-4.947726,4.426065,blues


In [7]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
data.head()

,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.391059,0.137728,1811.076084,2052.332563,3927.809582,0.075480,-145.434570,102.829018,-12.517674,56.993469,...,7.457216,-10.470443,-2.360484,-6.783623,2.671134,-4.760879,-0.949005,0.024832,-2.005315,blues
1,0.338896,0.251350,2141.461656,2168.015560,4627.997015,0.105151,-29.362093,108.667946,-25.573164,38.429470,...,5.456504,-7.687712,7.410599,-11.319178,7.229288,-9.466551,1.930059,-6.328475,-1.304813,blues
2,0.341865,0.259714,2094.919420,2158.335963,4482.447895,0.100570,-30.094570,103.732292,-19.754238,56.404770,...,11.213472,-8.842510,9.090302,-3.579252,4.372715,-2.830806,4.725090,0.485058,1.475955,blues
3,0.291308,0.141796,1459.078483,1388.913312,2795.616429,0.073028,-190.149460,130.296951,-36.344139,33.013050,...,-3.125678,-6.593118,-9.942267,-6.537224,-10.064754,-10.912171,-6.972478,-3.449033,-6.495510,blues
4,0.365575,0.219080,2645.681661,2426.198512,5612.507729,0.136469,-52.693062,74.430283,-18.692436,56.323868,...,7.792499,-10.561926,6.946956,-9.127866,13.020091,-9.854002,3.689460,-4.947726,4.426065,blues


In [8]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [10]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [20]:
len(X_train)

800

In [12]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [13]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
7/7 [==============================] - 0s 1ms/step - loss: 2.2463 - accuracy: 0.1424
Epoch 2/20
7/7 [==============================] - 0s 2ms/step - loss: 1.8997 - accuracy: 0.3625
Epoch 3/20
7/7 [==============================] - 0s 2ms/step - loss: 1.6771 - accuracy: 0.3753
Epoch 4/20
7/7 [==============================] - 0s 2ms/step - loss: 1.5009 - accuracy: 0.4638
Epoch 5/20
7/7 [==============================] - 0s 3ms/step - loss: 1.3228 - accuracy: 0.5437
Epoch 6/20
7/7 [==============================] - 0s 3ms/step - loss: 1.2259 - accuracy: 0.5746
Epoch 7/20
7/7 [==============================] - 0s 3ms/step - loss: 1.1156 - accuracy: 0.6197
Epoch 8/20
7/7 [==============================] - 0s 3ms/step - loss: 1.0384 - accuracy: 0.6584
Epoch 9/20
7/7 [==============================] - 0s 3ms/step - loss: 0.9924 - accuracy: 0.6747
Epoch 10/20
7/7 [==============================] - 0s 2ms/step - loss: 0.8738 - accuracy: 0.7279
Epoch 11/20
7/7 [======================

In [15]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

7/7 [==============================] - 0s 643us/step - loss: 1.0728 - accuracy: 0.6300
test_acc:  0.6299999952316284


In [16]:
predictions = model.predict(X_test)
np.argmax(predictions[0])

3

In [21]:
predictions

array([[1.5611175e-01, 9.6833846e-03, 4.7537502e-02, ..., 2.0929160e-02,
        2.2913009e-02, 9.7172163e-02],
       [4.4338875e-02, 3.8081703e-03, 2.5923138e-02, ..., 6.5722549e-04,
        5.1715553e-01, 2.8299674e-01],
       [4.4180360e-03, 4.8684124e-03, 9.3383305e-02, ..., 2.7199900e-01,
        1.0062981e-01, 3.0174154e-01],
       ...,
       [8.4614544e-04, 5.8595562e-01, 3.4280678e-03, ..., 1.1211846e-04,
        3.4054623e-03, 8.8868733e-04],
       [7.5222415e-01, 1.5942636e-04, 1.2088443e-01, ..., 4.7028872e-05,
        7.5483485e-04, 2.2549576e-03],
       [2.8059769e-02, 7.0485659e-03, 2.1351677e-01, ..., 8.4081216e-04,
        3.0906898e-01, 1.6977008e-01]], dtype=float32)